<h3>Step 1: Create the raster product processing file in the raster_product_development/products folder</h3>

<h3>Step 2: Tweak the following variable per the service you are developing</h3>

In [ ]:
SERVICE_NAME = 'mrf_gfs_10day_snow_water_equivalent_change'
PRODUCT_FILE_NAME = SERVICE_NAME  # The name of the file in the raster_product_development/products folder. It's not always the same as the service name.
FILE_FORMAT = 'common/data/model/com/nwm/prod/nwm.{{datetime:%Y%m%d}}/medium_range_mem1/nwm.t{{datetime:%H}}z.medium_range.land_1.f003.conus.nc'
FILE_STEP = '24H'
FILE_WINDOW = 'P10D'
REFERENCE_TIME = '2024-06-13 00:00:00'
RUN_OPTIMIZE_RASTERS_STEP = True  # Set this to true when you are ready to create Pro Project for service

<h3>Step 3: Click the "Restart Kernel and Run All Cells..." icon above (i.e. the double play icon) to test/execute your script</h3>

When processed successfully, you will see "ALL DONE!" printed below and your raster outputs will be found at s3://hydrovis-ti-fim-us-east-1/processing_outputs/<SERVICE_NAME>/YYYYMMDD/HH/workspace/

In [ ]:
REFERENCE_TIME_DATE = ''.join(REFERENCE_TIME.split(' ')[0].split('-'))
REFERENCE_TIME_HOUR = REFERENCE_TIME.split(' ')[1].split(':')[0]

# The name of the product file as found in the raster_product_development/products folder (often the same as the product/service name)
RASTER_PROCESSING_EVENT_METADATA = {
    "reference_time": REFERENCE_TIME,
    "product": {
        "product": SERVICE_NAME,
        "raster_input_files": {
            "file_format": FILE_FORMAT,
            "file_step": FILE_STEP,
            "file_window": FILE_WINDOW,
            "product_file": PRODUCT_FILE_NAME,
            "bucket": "nws-shared-data-226711853580-us-east-1"
        },
        "raster_outputs": {
            "output_bucket": "hydrovis-ti-fim-us-east-1",
            "output_raster_workspaces": [
                {
                    f"{SERVICE_NAME}": f"processing_outputs/{SERVICE_NAME}/{REFERENCE_TIME_DATE}/{REFERENCE_TIME_HOUR}/workspace"
                }
            ]
        }
    }
}

In [ ]:
!pip install isodate
from raster_product_development.lambda_function import lambda_handler as run_processing

print('Running processing...')
output = run_processing(RASTER_PROCESSING_EVENT_METADATA, None)
print('... Done!')
if RUN_OPTIMIZE_RASTERS_STEP:
    import boto3
    import botocore
    import json
    boto_config = botocore.client.Config(max_pool_connections=1, connect_timeout=60, read_timeout=600)
    lambda_client = boto3.client('lambda', config=boto_config)
    print('Optimizing output rasters...')
    for output_raster in output['output_rasters']['output_rasters']:
        lambda_client.invoke(
            FunctionName='hv-vpp-ti-viz-optimize-rasters',
            InvocationType='Event',
            Payload=json.dumps({
                "output_raster": output_raster,
                "output_bucket": output['output_rasters']['output_bucket']
            })
        )
    print('... Done!')
print('ALL DONE!')

<h3>Step 4: Create a Pro Projecto to symbolize the newly created raster data</h3>

Instructions on connecting your ArcGIS Pro to the raster data in S3 data can be found here: https://docs.google.com/presentation/d/1i7-QZhW7DUg3AV9davUy_m4pRutH0nD1iWCMILBm31I/edit#slide=id.p

<h3>Step 5: Save .mapx file of your ArcGIS Pro Project and also create service configuration .yml file</h3>

Commit these two files to your branch of the repo by creating a subfolder for your service here: https://github.com/NOAA-OWP/hydrovis/tree/ti/Core/LAMBDA/viz_functions/viz_publish_service/services

<h3>Step 6: Commit your product script to your branch of the repo</h3>

That script should go here: https://github.com/NOAA-OWP/hydrovis/tree/ti/Core/LAMBDA/viz_functions/image_based/viz_raster_processing/products

<h3>Step 7: Create a pipeilne product_config .yml file and commit it to your branch of the repo</h3>

That script should go here: https://github.com/NOAA-OWP/hydrovis/tree/ti/Core/LAMBDA/viz_functions/viz_initialize_pipeline/product_configs

<h3>Step 8: Push your branch of the repo and create/submit a Pull Request</h3>